# Day 9

## Part One

You've just said goodbye to the rebooted rover and left Mars when you receive a faint distress signal coming from the asteroid belt. It must be the Ceres monitoring station!

In order to lock on to the signal, you'll need to boost your sensors. The Elves send up the latest BOOST program - Basic Operation Of System Test.

While BOOST (your puzzle input) is capable of boosting your sensors, for tenuous safety reasons, it refuses to do so until the computer it runs on passes some checks to demonstrate it is a complete Intcode computer.

Your existing Intcode computer is missing one key feature: it needs support for parameters in relative mode.

Parameters in mode 2, relative mode, behave very similarly to parameters in position mode: the parameter is interpreted as a position. Like position mode, parameters in relative mode can be read from or written to.

The important difference is that relative mode parameters don't count from address 0. Instead, they count from a value called the relative base. The relative base starts at 0.

The address a relative mode parameter refers to is itself plus the current relative base. When the relative base is 0, relative mode parameters and position mode parameters with the same value refer to the same address.

For example, given a relative base of 50, a relative mode parameter of -7 refers to memory address 50 + -7 = 43.

The relative base is modified with the relative base offset instruction:

Opcode 9 adjusts the relative base by the value of its only parameter. The relative base increases (or decreases, if the value is negative) by the value of the parameter.
For example, if the relative base is 2000, then after the instruction 109,19, the relative base would be 2019. If the next instruction were 204,-34, then the value at address 1985 would be output.

Your Intcode computer will also need a few other capabilities:

The computer's available memory should be much larger than the initial program. Memory beyond the initial program starts with the value 0 and can be read or written like any other memory. (It is invalid to try to access memory at a negative address, though.)
The computer should have support for large numbers. Some instructions near the beginning of the BOOST program will verify this capability.
Here are some example programs that use these features:

109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99 takes no input and produces a copy of itself as output.
1102,34915192,34915192,7,4,7,99,0 should output a 16-digit number.
104,1125899906842624,99 should output the large number in the middle.
The BOOST program will ask for a single input; run it in test mode by providing it the value 1. It will perform a series of checks on each opcode, output any opcodes (and the associated parameter modes) that seem to be functioning incorrectly, and finally output a BOOST keycode.

Once your Intcode computer is fully functional, the BOOST program should report no malfunctioning opcodes when run in test mode; it should only output a single value, the BOOST keycode. What BOOST keycode does it produce?

To begin, get your puzzle input.

In [35]:
# Puzzle Input

intcode = list(map(int, open('input.txt', 'r').read().split(',')))

In [36]:
print(intcode)

[1102, 34463338, 34463338, 63, 1007, 63, 34463338, 63, 1005, 63, 53, 1101, 0, 3, 1000, 109, 988, 209, 12, 9, 1000, 209, 6, 209, 3, 203, 0, 1008, 1000, 1, 63, 1005, 63, 65, 1008, 1000, 2, 63, 1005, 63, 904, 1008, 1000, 0, 63, 1005, 63, 58, 4, 25, 104, 0, 99, 4, 0, 104, 0, 99, 4, 17, 104, 0, 99, 0, 0, 1101, 0, 252, 1023, 1101, 0, 0, 1020, 1102, 1, 39, 1013, 1102, 1, 234, 1029, 1102, 26, 1, 1016, 1101, 37, 0, 1005, 1101, 0, 27, 1011, 1101, 21, 0, 1000, 1101, 0, 29, 1019, 1101, 35, 0, 1003, 1102, 22, 1, 1007, 1102, 1, 32, 1001, 1101, 1, 0, 1021, 1102, 1, 216, 1027, 1102, 30, 1, 1012, 1102, 1, 24, 1009, 1101, 36, 0, 1002, 1101, 0, 31, 1010, 1101, 0, 243, 1028, 1102, 787, 1, 1024, 1102, 255, 1, 1022, 1102, 33, 1, 1017, 1102, 1, 23, 1004, 1102, 778, 1, 1025, 1102, 1, 28, 1008, 1101, 0, 223, 1026, 1102, 1, 25, 1015, 1101, 0, 20, 1006, 1102, 34, 1, 1014, 1101, 38, 0, 1018, 109, -4, 1202, 5, 1, 63, 1008, 63, 32, 63, 1005, 63, 203, 4, 187, 1106, 0, 207, 1001, 64, 1, 64, 1002, 64, 2, 64, 109, 37, 

In [37]:
#Code from Day 7
import numpy as np

def parse_long_direction(direction):
    shortopcode = str(direction).zfill(5)
    param_1_mode, param_2_mode, param_3_mode, opcode = int(shortopcode[2]), int(shortopcode[1]), int(shortopcode[0]), int(shortopcode[-2:])
    direction_list = [opcode, param_1_mode, param_2_mode, param_3_mode]
    return direction_list

In [166]:
def diagnosis_code_boost(program, input_value = 1):
    list_of_zeros = [0] * 1000000
    program_copy = program + list_of_zeros
    opcode_index = 0
    relative_base = 0
    while True:
        opcode = program_copy[opcode_index]
        direction = parse_long_direction(opcode)[0]
        elements = []
        if direction in {1, 2, 7, 8}:
            elements_length = 3
        elif direction in {3, 4, 9}:
            elements_length = 1
        elif direction in {5,6}:
            elements_length = 2
        for i in range(1,elements_length+1):
            if parse_long_direction(opcode)[i] == 0:
                elements.append(program_copy[opcode_index + i])
            elif parse_long_direction(opcode)[i] == 1:
                elements.append(opcode_index + i)
            elif parse_long_direction(opcode)[i] == 2:
                #elements.append(program_copy[opcode_index + i + relative_base])
                elements.append(program_copy[opcode_index + i] + relative_base)
        if direction == 99:
            return output   
        elif direction == 1:
            program_copy[elements[2]] = program_copy[elements[0]] + program_copy[elements[1]]
            opcode_index +=4
        elif direction == 2:
            program_copy[elements[2]] = program_copy[elements[0]] * program_copy[elements[1]]
            opcode_index +=4
        elif direction == 3:
            program_copy[elements[0]] = input_value
            opcode_index +=2
        elif direction == 4:
            output = program_copy[elements[0]]
            print(f"output is {output}")
            print("direction is 4")
            opcode_index +=2
        elif direction == 5:
            if program[elements[0]] != 0:
                opcode_index = program_copy[elements[1]]
            else:
                opcode_index +=3
        elif direction == 6:
            if program[elements[0]] == 0:
                opcode_index = program_copy[elements[1]]
            else:
                opcode_index +=3
        elif direction == 7:
            if program_copy[elements[0]] < program_copy[elements[1]]:
                program_copy[elements[2]] = 1
                opcode_index +=4
            else:
                program_copy[elements[2]] = 0
                opcode_index +=4
        elif direction == 8:
            if program_copy[elements[0]] == program_copy[elements[1]]:
                program_copy[elements[2]] = 1
                opcode_index +=4
            else:
                program_copy[elements[2]] = 0
                opcode_index +=4
        elif direction == 9:
            relative_base += program_copy[elements[0]]
            #relative_base += elements[0]
            print(f"the relative base is {relative_base}")
            opcode_index +=2


In [167]:
diagnosis_code_boost(intcode, input_value=1)

the relative base is 988
the relative base is 991
the relative base is 994
the relative base is 997
the relative base is 1000
output is 203
direction is 4
output is 0
direction is 4


0

In [152]:
testing_boost(intcode)

the relative base is 988
the relative base is 2090
the relative base is 2093
the relative base is 3195
the relative base is 4297
output is 203
direction is 4
output is 0
direction is 4


IndexError: list index out of range

In [162]:
test_intcode = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
add_zeros = [0] * 100000
test_intcode = test_intcode + add_zeros
print(test_intcode)
diagnosis_code_boost(test_intcode)

[109, 1, 204, -1, 1001, 100, 1, 100, 1008, 100, 16, 101, 1006, 101, 0, 99, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

direction is 4
the relative base is 308
output is 109
direction is 4
the relative base is 309
output is 109
direction is 4
the relative base is 310
output is 109
direction is 4
the relative base is 311
output is 109
direction is 4
the relative base is 312
output is 109
direction is 4
the relative base is 313
output is 109
direction is 4
the relative base is 314
output is 109
direction is 4
the relative base is 315
output is 109
direction is 4
the relative base is 316
output is 109
direction is 4
the relative base is 317
output is 109
direction is 4
the relative base is 318
output is 109
direction is 4
the relative base is 319
output is 109
direction is 4
the relative base is 320
output is 109
direction is 4
the relative base is 321
output is 109
direction is 4
the relative base is 322
output is 109
direction is 4
the relative base is 323
output is 109
direction is 4
the relative base is 324
output is 109
direction is 4
the relative base is 325
output is 109
direction is 4
the relative 

direction is 4
the relative base is 641
output is 109
direction is 4
the relative base is 642
output is 109
direction is 4
the relative base is 643
output is 109
direction is 4
the relative base is 644
output is 109
direction is 4
the relative base is 645
output is 109
direction is 4
the relative base is 646
output is 109
direction is 4
the relative base is 647
output is 109
direction is 4
the relative base is 648
output is 109
direction is 4
the relative base is 649
output is 109
direction is 4
the relative base is 650
output is 109
direction is 4
the relative base is 651
output is 109
direction is 4
the relative base is 652
output is 109
direction is 4
the relative base is 653
output is 109
direction is 4
the relative base is 654
output is 109
direction is 4
the relative base is 655
output is 109
direction is 4
the relative base is 656
output is 109
direction is 4
the relative base is 657
output is 109
direction is 4
the relative base is 658
output is 109
direction is 4
the relative 

the relative base is 974
output is 109
direction is 4
the relative base is 975
output is 109
direction is 4
the relative base is 976
output is 109
direction is 4
the relative base is 977
output is 109
direction is 4
the relative base is 978
output is 109
direction is 4
the relative base is 979
output is 109
direction is 4
the relative base is 980
output is 109
direction is 4
the relative base is 981
output is 109
direction is 4
the relative base is 982
output is 109
direction is 4
the relative base is 983
output is 109
direction is 4
the relative base is 984
output is 109
direction is 4
the relative base is 985
output is 109
direction is 4
the relative base is 986
output is 109
direction is 4
the relative base is 987
output is 109
direction is 4
the relative base is 988
output is 109
direction is 4
the relative base is 989
output is 109
direction is 4
the relative base is 990
output is 109
direction is 4
the relative base is 991
output is 109
direction is 4
the relative base is 992
out

output is 109
direction is 4
the relative base is 1141
output is 109
direction is 4
the relative base is 1142
output is 109
direction is 4
the relative base is 1143
output is 109
direction is 4
the relative base is 1144
output is 109
direction is 4
the relative base is 1145
output is 109
direction is 4
the relative base is 1146
output is 109
direction is 4
the relative base is 1147
output is 109
direction is 4
the relative base is 1148
output is 109
direction is 4
the relative base is 1149
output is 109
direction is 4
the relative base is 1150
output is 109
direction is 4
the relative base is 1151
output is 109
direction is 4
the relative base is 1152
output is 109
direction is 4
the relative base is 1153
output is 109
direction is 4
the relative base is 1154
output is 109
direction is 4
the relative base is 1155
output is 109
direction is 4
the relative base is 1156
output is 109
direction is 4
the relative base is 1157
output is 109
direction is 4
the relative base is 1158
output is 

...

direction is 4
the relative base is 9135
output is 109
direction is 4
the relative base is 9136
output is 109
direction is 4
the relative base is 9137
output is 109
direction is 4
the relative base is 9138
output is 109
direction is 4
the relative base is 9139
output is 109
direction is 4
the relative base is 9140
output is 109
direction is 4
the relative base is 9141
output is 109
direction is 4
the relative base is 9142
output is 109
direction is 4
the relative base is 9143
output is 109
direction is 4
the relative base is 9144
output is 109
direction is 4
the relative base is 9145
output is 109
direction is 4
the relative base is 9146
output is 109
direction is 4
the relative base is 9147
output is 109
direction is 4
the relative base is 9148
output is 109
direction is 4
the relative base is 9149
output is 109
direction is 4
the relative base is 9150
output is 109
direction is 4
the relative base is 9151
output is 109
direction is 4
the relative base is 9152
output is 109
direction 

output is 109
direction is 4
the relative base is 9302
output is 109
direction is 4
the relative base is 9303
output is 109
direction is 4
the relative base is 9304
output is 109
direction is 4
the relative base is 9305
output is 109
direction is 4
the relative base is 9306
output is 109
direction is 4
the relative base is 9307
output is 109
direction is 4
the relative base is 9308
output is 109
direction is 4
the relative base is 9309
output is 109
direction is 4
the relative base is 9310
output is 109
direction is 4
the relative base is 9311
output is 109
direction is 4
the relative base is 9312
output is 109
direction is 4
the relative base is 9313
output is 109
direction is 4
the relative base is 9314
output is 109
direction is 4
the relative base is 9315
output is 109
direction is 4
the relative base is 9316
output is 109
direction is 4
the relative base is 9317
output is 109
direction is 4
the relative base is 9318
output is 109
direction is 4
the relative base is 9319
output is 

output is 109
direction is 4
the relative base is 9635
output is 109
direction is 4
the relative base is 9636
output is 109
direction is 4
the relative base is 9637
output is 109
direction is 4
the relative base is 9638
output is 109
direction is 4
the relative base is 9639
output is 109
direction is 4
the relative base is 9640
output is 109
direction is 4
the relative base is 9641
output is 109
direction is 4
the relative base is 9642
output is 109
direction is 4
the relative base is 9643
output is 109
direction is 4
the relative base is 9644
output is 109
direction is 4
the relative base is 9645
output is 109
direction is 4
the relative base is 9646
output is 109
direction is 4
the relative base is 9647
output is 109
direction is 4
the relative base is 9648
output is 109
direction is 4
the relative base is 9649
output is 109
direction is 4
the relative base is 9650
output is 109
direction is 4
the relative base is 9651
output is 109
direction is 4
the relative base is 9652
output is 

direction is 4
the relative base is 9968
output is 109
direction is 4
the relative base is 9969
output is 109
direction is 4
the relative base is 9970
output is 109
direction is 4
the relative base is 9971
output is 109
direction is 4
the relative base is 9972
output is 109
direction is 4
the relative base is 9973
output is 109
direction is 4
the relative base is 9974
output is 109
direction is 4
the relative base is 9975
output is 109
direction is 4
the relative base is 9976
output is 109
direction is 4
the relative base is 9977
output is 109
direction is 4
the relative base is 9978
output is 109
direction is 4
the relative base is 9979
output is 109
direction is 4
the relative base is 9980
output is 109
direction is 4
the relative base is 9981
output is 109
direction is 4
the relative base is 9982
output is 109
direction is 4
the relative base is 9983
output is 109
direction is 4
the relative base is 9984
output is 109
direction is 4
the relative base is 9985
output is 109
direction 

KeyboardInterrupt: 

In [151]:
# Puzzle Input
def testing_boost(program):
    newintcode, output = diagnosis_code_boost_test(program, input_value = 1)
    cooldude, output2 = diagnosis_code_boost_test(newintcode, input_value = 1)
    print(output2)
    